# Data Loading and Preprocessing

In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define paths
DATASET_PATH = r"C:\Users\Administrator\Documents\Sunny's ALU\Heimdall\FacePix"
CSV_PATH = r"C:\Users\Administrator\Documents\Sunny's ALU\Heimdall\facepix_metadata_named.csv"

# Load metadata
df = pd.read_csv(CSV_PATH)

# Keep only valid file paths
df = df[df['file_path'].apply(os.path.exists)].reset_index(drop=True)

# Encode names into numeric labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['person_name'])

# Train-val-test split
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Load grayscale images
def load_images(df_subset):
    X, y = [], []
    for _, row in df_subset.iterrows():
        img = cv2.imread(row['file_path'], cv2.IMREAD_GRAYSCALE)
        img = img / 255.0  # normalize
        X.append(img.flatten())
        y.append(row['label'])
    return np.array(X), np.array(y)

X_train, y_train = load_images(train_df)
X_val, y_val = load_images(val_df)
X_test, y_test = load_images(test_df)

print("Input shape:", X_train.shape[1])  # 3060
print("Number of Classes:", len(np.unique(y_train)))

Input shape: 3060
Number of Classes: 30


# Baseline Neural Network Model (No Optimization)

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

# Create baseline model without optimization
model1 = Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(30, activation='softmax')
])

model1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model1.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), verbose=0)

y_pred = model1.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

acc = accuracy_score(y_test, y_pred_classes)
prec = precision_score(y_test, y_pred_classes, average='macro')
rec = recall_score(y_test, y_pred_classes, average='macro')
f1 = f1_score(y_test, y_pred_classes, average='macro')
loss = log_loss(y_test, y_pred)

# Print the Reults
print("Baseline Model Results")
print("Accuracy:", acc, "| Precision:", prec, "| Recall:", rec, "| F1:", f1, "| Loss:", loss)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Baseline Model Results
Accuracy: 0.0785276073619632 | Precision: 0.030991325428711993 | Recall: 0.07901234567901234 | F1: 0.030284906581624083 | Loss: 2.990606810185863


C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


# Neural Network with Optimization Techniques

In [3]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Optimized Model (Dropout, EarlyStopping, RMSprop)
model2 = Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(30, activation='softmax')
])

early_stop2 = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
               loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model2.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), callbacks=[early_stop2], verbose=0)

y_pred2 = model2.predict(X_test)
y_pred_classes2 = np.argmax(y_pred2, axis=1)

# Print the Results
print("Optimized Model (RMSprop + Dropout + EarlyStopping)")
print("Accuracy:", accuracy_score(y_test, y_pred_classes2))

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Optimized Model (RMSprop + Dropout + EarlyStopping)
Accuracy: 0.19386503067484662


# Deeper Neural Network with Adam + Dropout + EarlyStopping

In [4]:
# DNN with Adam+Dropout+EarlyStopping
model3 = Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(30, activation='softmax')
])

early_stop3 = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
               loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model3.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val), callbacks=[early_stop3], verbose=0)

y_pred3 = model3.predict(X_test)
y_pred_classes3 = np.argmax(y_pred3, axis=1)

# Print the Result
print("Deep Optimized Model (Adam)")
print("F1 Score:", f1_score(y_test, y_pred_classes3, average='macro'))

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Deep Optimized Model (Adam)
F1 Score: 0.0023106546854942235


# Shallow Optimized Model with SGD and Dropout

In [5]:
model4 = Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(30, activation='softmax')
])

early_stop4 = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model4.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
               loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model4.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop4], verbose=0)

y_pred4 = model4.predict(X_test)
y_pred_classes4 = np.argmax(y_pred4, axis=1)

# Print the Result
print("Optimized Model with SGD")
print("Accuracy:", accuracy_score(y_test, y_pred_classes4))

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Optimized Model with SGD
Accuracy: 0.03680981595092025


# Classical ML Model – Logistic Regression (Optimized)

In [6]:
from sklearn.linear_model import LogisticRegression
import joblib

clf = LogisticRegression(max_iter=1000, solver='lbfgs', C=1.0, multi_class='multinomial')
clf.fit(X_train, y_train)

y_pred_lr = clf.predict(X_test)
y_proba_lr = clf.predict_proba(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("F1 Score:", f1_score(y_test, y_pred_lr, average='macro'))
print("Loss:", log_loss(y_test, y_proba_lr))

joblib.dump(clf, "saved_models/logistic_regression_model.pkl")

C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Accuracy: 1.0
F1 Score: 1.0
Loss: 0.05018911572133783


['saved_models/logistic_regression_model.pkl']

# Hyperparameter Tuning for Logistic Regression

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define parameter grid
param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'solver': ['lbfgs', 'saga'],
    'multi_class': ['multinomial'],
    'max_iter': [1000]
}

# Setup GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found:", grid_search.best_params_)

# Retrain with best parameters
best_log_reg = grid_search.best_estimator_
best_log_reg.fit(X_train, y_train)

# Evaluate
y_pred_lr = best_log_reg.predict(X_test)
y_proba_lr = best_log_reg.predict_proba(X_test)

acc = accuracy_score(y_test, y_pred_lr)
prec = precision_score(y_test, y_pred_lr, average='macro')
rec = recall_score(y_test, y_pred_lr, average='macro')
f1 = f1_score(y_test, y_pred_lr, average='macro')
loss = log_loss(y_test, y_proba_lr)

print(f"Tuned Logistic Regression\nAccuracy: {acc}\nPrecision: {prec}\nRecall: {rec}\nF1: {f1}\nLoss: {loss}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits


C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique clas

C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_response.py:203: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  target_type = type_of_target(classes)
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always

Best parameters found: {'C': 10.0, 'max_iter': 1000, 'multi_class': 'multinomial', 'solver': 'lbfgs'}


C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Tuned Logistic Regression
Accuracy: 0.9987730061349693
Precision: 0.9988505747126436
Recall: 0.9987654320987654
F1: 0.9987862738607525
Loss: 0.014344575834673556


# Result Summary Table

In [8]:
results = [
    ["instance_1", "Adam", 10, False, log_loss(y_test, model1.predict(X_test)), accuracy_score(y_test, y_pred_classes), precision_score(y_test, y_pred_classes, average='macro'), recall_score(y_test, y_pred_classes, average='macro'), f1_score(y_test, y_pred_classes, average='macro')],
    ["instance_2", "RMSprop", 30, True, log_loss(y_test, y_pred2), accuracy_score(y_test, y_pred_classes2), precision_score(y_test, y_pred_classes2, average='macro'), recall_score(y_test, y_pred_classes2, average='macro'), f1_score(y_test, y_pred_classes2, average='macro')],
    ["instance_3", "Adam", 40, True, log_loss(y_test, y_pred3), accuracy_score(y_test, y_pred_classes3), precision_score(y_test, y_pred_classes3, average='macro'), recall_score(y_test, y_pred_classes3, average='macro'), f1_score(y_test, y_pred_classes3, average='macro')],
    ["instance_4", "SGD", 50, True, log_loss(y_test, y_pred4), accuracy_score(y_test, y_pred_classes4), precision_score(y_test, y_pred_classes4, average='macro'), recall_score(y_test, y_pred_classes4, average='macro'), f1_score(y_test, y_pred_classes4, average='macro')],
    ["LogisticRegression", "N/A", "N/A", False, log_loss(y_test, y_proba_lr), accuracy_score(y_test, y_pred_lr), precision_score(y_test, y_pred_lr, average='macro'), recall_score(y_test, y_pred_lr, average='macro'), f1_score(y_test, y_pred_lr, average='macro')],
    ["LogisticRegression_Tuned", "N/A", "N/A", False, loss, acc, prec, rec, f1]
]

results_df = pd.DataFrame(results, columns=["Instance", "Optimizer", "Epochs", "EarlyStopping", "Loss", "Accuracy", "Precision", "Recall", "F1-score"])
results_df.to_csv("training_results.csv", index=False)
results_df

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod

,Instance,Optimizer,Epochs,EarlyStopping,Loss,Accuracy,Precision,Recall,F1-score
0,instance_1,Adam,10,False,2.990607,0.078528,0.030991,0.079012,0.030285
1,instance_2,RMSprop,30,True,2.716355,0.193865,0.185908,0.192725,0.137108
2,instance_3,Adam,40,True,3.398325,0.033129,0.001197,0.033333,0.002311
3,instance_4,SGD,50,True,3.399580,0.036810,0.004076,0.037037,0.006139
4,LogisticRegression,N/A,N/A,False,0.014345,0.998773,0.998851,0.998765,0.998786
5,LogisticRegression_Tuned,N/A,N/A,False,0.014345,0.998773,0.998851,0.998765,0.998786
